In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.datasets import load_digits
from sklearn.datasets import load_iris

set_config(print_changed_only=False)


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

%config InlineBackend.figure_format='svg'
plt.rcParams['figure.dpi']=120

pd.options.display.float_format='{:,.2f}'.format
pd.set_option('display.max_colwidth', None)


## Overfitting issue is the very common issue in the field of machine learning and <b style='color:blue'>L1</b>and<b style='color:blue'>L2 regularizations</b>  are the techniques that can be used to <b style='color:blue; '> address the overfitting issue. </b> How to address the overfit model and improve the accuracy.

## When trying to make an algorithm equation better fit, the equation becomes more  complex. Regularization is a technique which gets the equation simpler


<img src='./data/regularization.jpg' width=800 height=500>
<img src='./data/regularization_1.jpg' width=800 height=500>



<h1 align='center'> <mark style='color:yellow; background-color:black' >How to reduce overfitting</mark></h1>

<img src='./data/regularization_2.jpg' width=800 height=500>
<img src='./data/regularization_3.jpg' width=600 height=300>
<img src='./data/regularization_4.jpg' width=1000 height=600>

**We are going to use Melbourne House Price Dataset where we'll predict House Predictions based on various features.** <br>
<a href="https://www.kaggle.com/anthonypino/melbourne-housing-market"> data link</a>

In [2]:
# Suppress Warnings for clean notebook
import warnings
warnings.filterwarnings('ignore')

In [3]:
dataset = pd.read_csv('./data/Melbourne_housing_FULL.csv')

In [4]:
dataset.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.50,"3,067.00",...,1.00,1.00,126.00,NaN,NaN,Yarra City Council,-37.80,145.00,Northern Metropolitan,"4,019.00"
1,Abbotsford,85 Turner St,2,h,"1,480,000.00",S,Biggin,3/12/2016,2.50,"3,067.00",...,1.00,1.00,202.00,NaN,NaN,Yarra City Council,-37.80,145.00,Northern Metropolitan,"4,019.00"
2,Abbotsford,25 Bloomburg St,2,h,"1,035,000.00",S,Biggin,4/02/2016,2.50,"3,067.00",...,1.00,0.00,156.00,79.00,"1,900.00",Yarra City Council,-37.81,144.99,Northern Metropolitan,"4,019.00"
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.50,"3,067.00",...,2.00,1.00,0.00,NaN,NaN,Yarra City Council,-37.81,145.01,Northern Metropolitan,"4,019.00"
4,Abbotsford,5 Charles St,3,h,"1,465,000.00",SP,Biggin,4/03/2017,2.50,"3,067.00",...,2.00,0.00,134.00,150.00,"1,900.00",Yarra City Council,-37.81,144.99,Northern Metropolitan,"4,019.00"


In [5]:
dataset.nunique()

Suburb             351
Address          34009
Rooms               12
Type                 3
Price             2871
Method               9
SellerG            388
Date                78
Distance           215
Postcode           211
Bedroom2            15
Bathroom            11
Car                 15
Landsize          1684
BuildingArea       740
YearBuilt          160
CouncilArea         33
Lattitude        13402
Longtitude       14524
Regionname           8
Propertycount      342
dtype: int64

In [6]:
# let's use limited columns which makes more sense for serving our purpose
cols_to_use = ['Suburb', 'Rooms', 'Type', 'Method', 'SellerG', 'Regionname', 'Propertycount', 
               'Distance', 'CouncilArea', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'Price']
dataset = dataset[cols_to_use]

In [7]:
dataset.head()

,Suburb,Rooms,Type,Method,SellerG,Regionname,Propertycount,Distance,CouncilArea,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Price
0,Abbotsford,2,h,SS,Jellis,Northern Metropolitan,"4,019.00",2.50,Yarra City Council,2.00,1.00,1.00,126.00,NaN,NaN
1,Abbotsford,2,h,S,Biggin,Northern Metropolitan,"4,019.00",2.50,Yarra City Council,2.00,1.00,1.00,202.00,NaN,"1,480,000.00"
2,Abbotsford,2,h,S,Biggin,Northern Metropolitan,"4,019.00",2.50,Yarra City Council,2.00,1.00,0.00,156.00,79.00,"1,035,000.00"
3,Abbotsford,3,u,VB,Rounds,Northern Metropolitan,"4,019.00",2.50,Yarra City Council,3.00,2.00,1.00,0.00,NaN,NaN
4,Abbotsford,3,h,SP,Biggin,Northern Metropolitan,"4,019.00",2.50,Yarra City Council,3.00,2.00,0.00,134.00,150.00,"1,465,000.00"


In [8]:
dataset.shape
dataset.isna().sum()

(34857, 15)

Suburb               0
Rooms                0
Type                 0
Method               0
SellerG              0
Regionname           3
Propertycount        3
Distance             1
CouncilArea          3
Bedroom2          8217
Bathroom          8226
Car               8728
Landsize         11810
BuildingArea     21115
Price             7610
dtype: int64

**Handling Missing Values**

In [9]:
# Some feature's missing values can be treated as zero (another class for NA values or absence of that feature)
# like 0 for Propertycount, Bedroom2 will refer to other class of NA values
# like 0 for Car feature will mean that there's no car parking feature with house
cols_to_fill_zero = ['Propertycount', 'Distance', 'Bedroom2', 'Bathroom', 'Car']
dataset[cols_to_fill_zero] = dataset[cols_to_fill_zero].fillna(0)

# other continuous features can be imputed with mean for faster results since our focus is on Reducing overfitting
# using Lasso and Ridge Regression
dataset['Landsize'] = dataset['Landsize'].fillna(dataset.Landsize.mean())
dataset['BuildingArea'] = dataset['BuildingArea'].fillna(dataset.BuildingArea.mean())

**Drop NA values of Price, since it's our predictive variable we won't impute it**

In [10]:
dataset.dropna(inplace=True)

In [11]:
dataset.shape

(27244, 15)

In [12]:
dataset.head()

,Suburb,Rooms,Type,Method,SellerG,Regionname,Propertycount,Distance,CouncilArea,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Price
1,Abbotsford,2,h,S,Biggin,Northern Metropolitan,"4,019.00",2.50,Yarra City Council,2.00,1.00,1.00,202.00,160.26,"1,480,000.00"
2,Abbotsford,2,h,S,Biggin,Northern Metropolitan,"4,019.00",2.50,Yarra City Council,2.00,1.00,0.00,156.00,79.00,"1,035,000.00"
4,Abbotsford,3,h,SP,Biggin,Northern Metropolitan,"4,019.00",2.50,Yarra City Council,3.00,2.00,0.00,134.00,150.00,"1,465,000.00"
5,Abbotsford,3,h,PI,Biggin,Northern Metropolitan,"4,019.00",2.50,Yarra City Council,3.00,2.00,1.00,94.00,160.26,"850,000.00"
6,Abbotsford,4,h,VB,Nelson,Northern Metropolitan,"4,019.00",2.50,Yarra City Council,3.00,1.00,2.00,120.00,142.00,"1,600,000.00"


**Let's one hot encode the categorical features**

In [13]:
dataset = pd.get_dummies(dataset, drop_first=True)
dataset.head()

,Rooms,Propertycount,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Price,Suburb_Aberfeldie,...,CouncilArea_Moorabool Shire Council,CouncilArea_Moreland City Council,CouncilArea_Nillumbik Shire Council,CouncilArea_Port Phillip City Council,CouncilArea_Stonnington City Council,CouncilArea_Whitehorse City Council,CouncilArea_Whittlesea City Council,CouncilArea_Wyndham City Council,CouncilArea_Yarra City Council,CouncilArea_Yarra Ranges Shire Council
1,2,"4,019.00",2.50,2.00,1.00,1.00,202.00,160.26,"1,480,000.00",0,...,0,0,0,0,0,0,0,0,1,0
2,2,"4,019.00",2.50,2.00,1.00,0.00,156.00,79.00,"1,035,000.00",0,...,0,0,0,0,0,0,0,0,1,0
4,3,"4,019.00",2.50,3.00,2.00,0.00,134.00,150.00,"1,465,000.00",0,...,0,0,0,0,0,0,0,0,1,0
5,3,"4,019.00",2.50,3.00,2.00,1.00,94.00,160.26,"850,000.00",0,...,0,0,0,0,0,0,0,0,1,0
6,4,"4,019.00",2.50,3.00,1.00,2.00,120.00,142.00,"1,600,000.00",0,...,0,0,0,0,0,0,0,0,1,0


**split the data into train and test dataset**

In [14]:
X = dataset.drop('Price', axis=1)
y = dataset['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

#### Let's train our Linear Regression Model on training dataset and check the accuracy on test set

In [15]:
reg = LinearRegression().fit(X_train, y_train)

reg.score(X_test, y_test)

reg.score(X_train, y_train)

0.13853683161537644

0.6827792395792723

**Here training score is 68% but test score is 13.85% which is very low**

Normal Regression is clearly overfitting the data, let's try other models

### Using Lasso (L1 Regularized) Regression Model

In [16]:
lasso_reg = Lasso(alpha=50, max_iter=100, tol=0.1)
lasso_reg.fit(X_train, y_train)

Lasso(alpha=50, copy_X=True, fit_intercept=True, max_iter=100,
      normalize='deprecated', positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.1, warm_start=False)

In [18]:
lasso_reg.score(X_test, y_test)

lasso_reg.score(X_train, y_train)

0.6636111369404489

0.6766985624766824

### Using Ridge (L2 Regularized) Regression Model

In [19]:
ridge_reg= Ridge(alpha=50, max_iter=100, tol=0.1)
ridge_reg.fit(X_train, y_train)

Ridge(alpha=50, copy_X=True, fit_intercept=True, max_iter=100,
      normalize='deprecated', positive=False, random_state=None, solver='auto',
      tol=0.1)

In [20]:
ridge_reg.score(X_train, y_train)

ridge_reg.score(X_train, y_train)

0.6622376739684328

0.6622376739684328

**We see that Lasso and Ridge Regularizations prove to be beneficial when our Simple Linear Regression Model overfits. These results may not be that contrast but significant in most cases.Also that L1 & L2 Regularizations are used in Neural Networks too**